<a href="https://colab.research.google.com/github/ruwanwija/Research-Models/blob/main/Industry_Research_Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase-admin

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your service account JSON file

Saving industry-research-a14f3-firebase-adminsdk-atpqc-3af3330e9c.json to industry-research-a14f3-firebase-adminsdk-atpqc-3af3330e9c.json


In [ ]:
import firebase_admin
from firebase_admin import credentials

# Replace 'path/to/your/serviceAccountKey.json' with the name of the uploaded file
cred = credentials.Certificate('industry-research-a14f3-firebase-adminsdk-atpqc-3af3330e9c.json')
firebase_admin.initialize_app(cred)


In [ ]:
import pandas as pd
import nltk
import re
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cluster import KMeans
import numpy as np
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase
cred = credentials.Certificate('industry-research-a14f3-firebase-adminsdk-atpqc-3af3330e9c.json')  # Replace with your JSON file path
firebase_admin.initialize_app(cred, name='Industry Research 3')

db = firestore.client()

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Preprocess the text
def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word not in stopwords.words('english')]
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        return ' '.join(tokens)
    else:
        return ''

# Load the dataset
df = pd.read_csv('Sample dataset II.csv')

# Preprocess the text data
df['preprocessed_text'] = df['reviews.text'].apply(preprocess_text)

# Define label mapping
labels = ["Room Services", "Location", "Food Quality", "Value for Money", "Comfort", "Staff Behavior"]

# Create an instance of MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Apply K-means clustering to generate labels (dummy labels for demonstration)
num_clusters = len(labels)  # Assuming we have 6 labels as defined above
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['preprocessed_text'])

# Fit KMeans model
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

# Add cluster labels to the DataFrame
df['cluster_label'] = kmeans.labels_

# Create a binary indicator for each cluster (multi-label format)
y = pd.get_dummies(df['cluster_label']).values

# Use StratifiedKFold to ensure stratified sampling
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store accuracies, F1 scores, and probabilities for each fold
accuracies = []
f1_scores = []
mean_probabilities = np.zeros(num_clusters)  # To store mean probabilities for each class

for train_index, test_index in stratified_kfold.split(X, np.argmax(y, axis=1)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create the SVM model
    base_model = svm.SVC(kernel='linear', probability=True)

    # Wrap the SVC model with OneVsRestClassifier to handle multi-label classification
    model = OneVsRestClassifier(base_model)

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # New record to predict
    new_record = ["Great place with unique room structure with wooden top. It helped spending our own time without any disturbance. We can see Beautiful Sun rise from the room by opening the curtain."]
    new_record_preprocessed = [preprocess_text(text) for text in new_record]
    new_record_tfidf = vectorizer.transform(new_record_preprocessed)

    # Predict the label for the new record
    predicted_labels_numeric = model.predict(new_record_tfidf)

    # Get the probabilities for each label category
    probabilities = model.predict_proba(new_record_tfidf)

    # Update mean probabilities
    mean_probabilities += probabilities[0]  # Sum up probabilities for averaging later

    # Display the predicted labels and probabilities
    probability_dict = {labels[i]: probabilities[0][i] for i in range(num_clusters)}

    # Find the label with the highest probability
    highest_label = max(probability_dict, key=probability_dict.get)
    highest_probability = probability_dict[highest_label]

    # Display results for the current fold
    print(f"Predicted Labels: {predicted_labels_numeric}")
    print("Probabilities for each class:")
    for label, prob in probability_dict.items():
        print(f"{label}: {prob:.4f}")  # Format to 4 decimal places

    print(f"\nHighest Probability Label: {highest_label} with probability: {highest_probability:.4f}")

    # Evaluate the model
    y_pred = model.predict(X_test)

    # Calculate accuracy and F1 score using binary arrays
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')

    # Store scores for the current fold
    accuracies.append(accuracy)
    f1_scores.append(f1)

# Calculate mean probabilities
mean_probabilities /= stratified_kfold.get_n_splits()  # Average the probabilities over folds

# Find the highest mean probability
highest_mean_label = labels[np.argmax(mean_probabilities)]
highest_mean_probability = np.max(mean_probabilities)

# Display overall results
print(f'\nMean Accuracy: {np.mean(accuracies)}')
print(f'Mean F1 Score: {np.mean(f1_scores)}')
print("\nMean Probabilities for each class:")
for label, mean_prob in zip(labels, mean_probabilities):
    print(f"{label}: {mean_prob:.4f}")

print(f"\nHighest Mean Probability Label: {highest_mean_label} with probability: {highest_mean_probability:.4f}")

# Save the highest mean probability label and the time to Firebase
data_to_store = {
    'label': highest_mean_label,
    'probability': highest_mean_probability,
    'timestamp': firestore.SERVER_TIMESTAMP  # Optional: include a timestamp
}

# Store in Firestore
doc_ref = db.collection('predictions').add(data_to_store)
print(f'Document written with ID: {doc_ref.id}')






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Predicted Labels: [[0 1 0 0 0 0]]
Probabilities for each class:
Room Services: 0.0031
Location: 0.8410
Food Quality: 0.0001
Value for Money: 0.0024
Comfort: 0.2531
Staff Behavior: 0.0031

Highest Probability Label: Location with probability: 0.8410
Predicted Labels: [[0 0 0 0 0 0]]
Probabilities for each class:
Room Services: 0.0023
Location: 0.6314
Food Quality: 0.0003
Value for Money: 0.0250
Comfort: 0.2129
Staff Behavior: 0.0013

Highest Probability Label: Location with probability: 0.6314
Predicted Labels: [[0 0 0 0 0 0]]
Probabilities for each class:
Room Services: 0.0027
Location: 0.6947
Food Quality: 0.0001
Value for Money: 0.0026
Comfort: 0.5097
Staff Behavior: 0.0080

Highest Probability Label: Location with probability: 0.6947
Predicted Labels: [[0 0 0 0 0 0]]
Probabilities for each class:
Room Services: 0.0025
Location: 0.6926
Food Quality: 0.0001
Value for Money: 0.0160
Comfort: 0.1789
Staff Behavior: 0.0033

Highest Probability Label: Location with probability: 0.6926
Pred

FailedPrecondition: 400 The Cloud Firestore API is not available for Firestore in Datastore Mode database projects/industry-research-a14f3/databases/(default).